# このコードの目的

少し複雑なネットワークをtensor.nnを使って簡単に定義する方法をまとめる。

In [1]:
import torch
import random

In [2]:
N = 1000
Dim_input = 5
Dim_output = 1

# toy dataを作成する。
# このデータは単純な線形結合では不可能でネットワークに工夫が必要
x = torch.rand((N, Dim_input))
y = 2 * x[:, 0] + 10 * x[:, 1] * x[:, 2]

x_train, y_train = x[:900], y[:900]
x_test, y_test = x[900:], y[900:]


In [3]:
# シンプルな解決法として、適切なactivation関数を設定した多層ネットワークを定義する
model = torch.nn.Sequential(
            torch.nn.Linear(Dim_input,  10, bias=None),
            torch.nn.ReLU(),
            torch.nn.Linear(10,  10, bias=None),
            torch.nn.ReLU(),
            torch.nn.Linear(10,  Dim_output, bias=None),
            )
criterion = torch.nn.L1Loss(reduction='mean') 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)

# 精度を最大化するために、batch_size=1で学習する。
batch_size = 1
epoch_size = 1000
for t in range(epoch_size):
    batch_indexes = random.choices(range(x_train.shape[0]), k=batch_size)
    _x, _y = x_train[batch_indexes], y_train[batch_indexes]
    loss = criterion(model(_x), _y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# test_setで学習結果を確認する    
for predict, answer in zip(model(x_test), y_test):
    print(predict, answer)

# ある程度は予測ができている。

tensor([2.6294], grad_fn=<SelectBackward>) tensor(2.3237)
tensor([1.8704], grad_fn=<SelectBackward>) tensor(2.0627)
tensor([2.5402], grad_fn=<SelectBackward>) tensor(1.8409)
tensor([2.1154], grad_fn=<SelectBackward>) tensor(2.1281)
tensor([1.5560], grad_fn=<SelectBackward>) tensor(1.4000)
tensor([5.3299], grad_fn=<SelectBackward>) tensor(7.2199)
tensor([0.6620], grad_fn=<SelectBackward>) tensor(0.9226)
tensor([5.4018], grad_fn=<SelectBackward>) tensor(6.9417)
tensor([2.6014], grad_fn=<SelectBackward>) tensor(2.0194)
tensor([1.5304], grad_fn=<SelectBackward>) tensor(1.3788)
tensor([4.3046], grad_fn=<SelectBackward>) tensor(4.1710)
tensor([2.5456], grad_fn=<SelectBackward>) tensor(1.2071)
tensor([3.4212], grad_fn=<SelectBackward>) tensor(2.8320)
tensor([3.4133], grad_fn=<SelectBackward>) tensor(2.2928)
tensor([1.6176], grad_fn=<SelectBackward>) tensor(0.4282)
tensor([2.0417], grad_fn=<SelectBackward>) tensor(1.7778)
tensor([6.3570], grad_fn=<SelectBackward>) tensor(8.3319)
tensor([3.2468

In [4]:
# toy data の生成モデルに等しいネットワーク構成になるようにmodelを組み合わせてみる。

simple_linear_model = torch.nn.Sequential(
            torch.nn.Linear(Dim_input,  Dim_output, bias=None),
            )
linear_cross_section_model = torch.nn.Sequential(
            torch.nn.Linear(Dim_input,  1, bias=None),
            )

criterion = torch.nn.L1Loss(reduction='mean') 
optimizer = torch.optim.Adam(
                            [
                                {'params':simple_linear_model.parameters()}, 
                                {'params':linear_cross_section_model.parameters()}
                            ]
                             , lr=0.01, weight_decay=0.01
            )

# 精度を最大化するために、batch_size=1で学習する。
batch_size = 1
epoch_size = 1000
for t in range(epoch_size):
    batch_indexes = random.choices(range(x_train.shape[0]), k=batch_size)
    _x, _y = x_train[batch_indexes], y_train[batch_indexes]
    _y_pre = simple_linear_model(_x) + linear_cross_section_model(_x)
    loss = criterion(_y_pre, _y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# test_setで学習結果を確認する    
for _x, answer in zip(x_test, y_test):
    predict = simple_linear_model(_x) + linear_cross_section_model(_x)
    print(predict, answer)

# ある程度は予測ができている。

tensor([2.4711], grad_fn=<AddBackward0>) tensor(2.3237)
tensor([1.5897], grad_fn=<AddBackward0>) tensor(2.0627)
tensor([2.2987], grad_fn=<AddBackward0>) tensor(1.8409)
tensor([2.0295], grad_fn=<AddBackward0>) tensor(2.1281)
tensor([1.4096], grad_fn=<AddBackward0>) tensor(1.4000)
tensor([5.0768], grad_fn=<AddBackward0>) tensor(7.2199)
tensor([0.5334], grad_fn=<AddBackward0>) tensor(0.9226)
tensor([5.0669], grad_fn=<AddBackward0>) tensor(6.9417)
tensor([2.4485], grad_fn=<AddBackward0>) tensor(2.0194)
tensor([1.2300], grad_fn=<AddBackward0>) tensor(1.3788)
tensor([3.9665], grad_fn=<AddBackward0>) tensor(4.1710)
tensor([2.6057], grad_fn=<AddBackward0>) tensor(1.2071)
tensor([3.1805], grad_fn=<AddBackward0>) tensor(2.8320)
tensor([3.1861], grad_fn=<AddBackward0>) tensor(2.2928)
tensor([1.4144], grad_fn=<AddBackward0>) tensor(0.4282)
tensor([1.8500], grad_fn=<AddBackward0>) tensor(1.7778)
tensor([6.0645], grad_fn=<AddBackward0>) tensor(8.3319)
tensor([3.0600], grad_fn=<AddBackward0>) tensor(